In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))
sys.path.append(os.path.join(os.getcwd(), "molgen"))
sys.path.append(os.path.join(os.getcwd(), "iclr19-graph2graph/props"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
# from molgen.dataloading.MolGraphBatchPreprocessor import MolGraph
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES


from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, load_checkpoint, initialize_models
from mol_opt.mol_opt import MolOpt
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.ot_utils import encode_target
from mol_opt.ot_utils import FGW 
from mol_opt.ot_utils import compute_barycenter

from molgen.dataloading.feat2smiles import feat2smiles
from molgen.dataloading.mol_drawer import MolDrawer
from molgen.metrics.Penalty import Penalty, RecPenalty
from molgen.metrics.mol_metrics import MolMetrics

from rdkit.Chem import MolFromSmiles
from tensorboardX import SummaryWriter

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import grad
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from molgen.metrics.Penalty import Penalty as PenaltyNew

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

/Users/octavdragoi/gitrepos/tum-thesis


In [13]:
model_type = "molemb"
sys.argv = ["", "-model_type", model_type, "-one_batch_train"]
args = get_args()
args.n_epochs = 100 
args.init_model = "{}-chembl1".format(model_type)
args.output_dir = "/Users/octavdragoi/Dropbox/tum-thesis/{}/".format(args.init_model)
args.tb_logs_dir = "/Users/octavdragoi/Dropbox/tum-thesis/logs_dev3/{}".format(args.init_model)
args.batch_size = 500 

In [3]:
model = args.init_model
model_iter = 16 

model_name = "model_{}_{}".format(model, model_iter)
print(args.output_dir + model_name)

molopt, molopt_decoder, _, pen, _, config, _ = load_checkpoint(args.output_dir + model_name ,init_fc = initialize_models, cpu = True)

metrics = MolMetrics(SYMBOLS, FORMAL_CHARGES, BOND_TYPES, False, device = 'cpu')
fgw_loss = FGW(alpha = 0.5)
# pen = PenaltyNew(config, model_iter)

molopt_module_list = torch.nn.ModuleList([molopt, molopt_decoder])

# molopt = MolOpt(args)
# molopt_decoder = MolOptDecoder(args)
print (molopt)
print (molopt_decoder)
print(molopt.args.model_type)
pen.log()

/Users/octavdragoi/Dropbox/tum-thesis/molemb-chembl1/model_molemb-chembl1_16
MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=200, bias=False)
    (W_message_h): Linear(in_features=200, out_features=200, bias=False)
    (W_message_o): Linear(in_features=293, out_features=150, bias=True)
    (W_mol_h): Linear(in_features=150, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=150, out_features=14000, bias=True)
  (opt1): Linear(in_features=14000, out_features=10500, bias=True)
)
MolOptDecoder(
  (fc1_SYMBOLS): Linear(in_features=150, out_features=150, bias=True)
  (fc2_SYMBOLS): Linear(in_features=150, out_features=64, bias=True)
  (fc1_CHARGES): Linear(in_features=150, out_features=150, bias=True)
  (fc2_CHARGES): Linear(in_features=150, out_features=5, bias=True)
  (fc1_BONDS): Linear

In [4]:
recpen = RecPenalty(args)

In [5]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, True)
datatype = "val_split"
train_data_loader = get_loader("molgen/data/chembl50", datatype, 50, same_number_atoms = True)

for i in train_data_loader:
#     X = (MolGraph(i[0]))
#     Y = (MolGraph(i[1]))
    X = MolGraph(i)
    Y = X
    break

In [6]:
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize_gumbel(*yhat_logits, tau = pen.tau)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

pen.mirror = ""

pen.tau = 0.05
fgw_loss_item = fgw_loss(*pred_pack, tau = 1)
print (fgw_loss_item.item()/args.batch_size)
pen_loss = pen(*pred_pack, model_iter)
print (pen_loss)
metrics.measure_batch(pred_pack[0], target)

/Users/octavdragoi/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


FGW torch.Size([23439, 5]) 0.000147338054375723
0.3594107818603516
(tensor(8.0064, grad_fn=<DivBackward0>), tensor(3.3244, grad_fn=<DivBackward0>), tensor(0.8394, grad_fn=<DivBackward0>))


({'batch_molecular_validity': 96.0,
  'batch_correctness': 0.0,
  'batch_symbol_accuracy': 98.01390268123139,
  'batch_molecular_disconnected_validity': 96.0,
  'batch_connected_components': 1.02,
  'batch_invalid_valency_nodes': 0.29791459781529295,
  'batch_nodes_0degree': 1,
  'batch_nodes_7plus_degree': 0,
  'invalid_euler_toofew': 0.0,
  'invalid_euler_toomany': 0.0},
 {'avg_euler_error': (-0.36, 50, 0.43040000000000006),
  'batch_node_degree': (1.9821251241310822, 1007, 0.724606010163217)})

In [7]:
x_embedding.shape

torch.Size([500, 70, 150])

In [8]:
mol_embedding = molopt.encode(X)

In [14]:
xhat_embedding = recpen(mol_embedding).view(args.batch_size, args.max_num_atoms, args.pc_hidden)

In [17]:
xhat_embedding.shape

torch.Size([500, 70, 150])

In [20]:
x_embedding.shape

torch.Size([500, 70, 150])

In [30]:
for idx, (st, le) in enumerate(X.scope):
    recpen.loss(x_embedding[idx,:le], xhat_embedding[idx,:le])

In [32]:
recpen.calculate_loss(x_embedding, xhat_embedding, X.scope)

tensor(115.6115, grad_fn=<MeanBackward0>)

In [34]:
for x in recpen.parameters():
    print (x)

Parameter containing:
tensor([[ 0.0519,  0.0592,  0.0685,  ..., -0.0065, -0.0630,  0.0449],
        [-0.0725, -0.0410,  0.0373,  ...,  0.0696,  0.0031,  0.0048],
        [-0.0535, -0.0358,  0.0290,  ..., -0.0292,  0.0568,  0.0433],
        ...,
        [ 0.0791, -0.0786, -0.0793,  ...,  0.0662,  0.0619, -0.0784],
        [-0.0589,  0.0470, -0.0022,  ..., -0.0074, -0.0285,  0.0492],
        [ 0.0277,  0.0193, -0.0569,  ...,  0.0493,  0.0252,  0.0788]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0540, -0.0172, -0.0362,  ..., -0.0435,  0.0807,  0.0447],
       requires_grad=True)
Parameter containing:
tensor([[-6.6515e-03,  5.0721e-04, -6.8333e-03,  ..., -3.6763e-03,
         -4.0087e-03,  7.3221e-04],
        [-2.5205e-03,  6.7956e-03, -3.7455e-03,  ..., -4.2382e-03,
         -3.7348e-03, -5.6535e-03],
        [-5.5820e-03,  6.1213e-03, -5.0610e-03,  ...,  1.9947e-03,
         -5.6051e-04,  5.6556e-03],
        ...,
        [-6.5154e-03, -4.6620e-03,  3.7811e-03,  ..., -